In [ ]:
import json
from typing import Optional

import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning import LightningDataModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split
from torchmetrics import MeanSquaredError, R2Score

In [ ]:
sparse_column = ["MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope",
                 "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl",
                 "Exterior1st", "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation", "BsmtQual",
                 "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", "CentralAir",
                 "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual",
                 "GarageCond", "PavedDrive", "PoolQC", "Fence", "MiscFeature", "SaleType", "SaleCondition"]

consistent_column = ["MSSubClass", "LotFrontage", "LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd",
                     "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF",
                     "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath",
                     "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageYrBlt", "GarageCars",
                     "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea",
                     "MiscVal", "MoSold", "YrSold"]

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.csv = pd.read_csv("./data/train.csv")
        with open("./data/sparse_label.json", "r", encoding="utf-8") as f:
            self.sparse_label = json.loads(f.read())
        with open("./data/sparse_column.json", "r", encoding="utf-8") as f:
            self.sparse_column = json.loads(f.read())

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, index):
        item = self.csv.iloc[index]
        feat = []
        for head in self.csv:
            if head == "Id" or head == "SalePrice":
                continue
            if head in self.sparse_label:
                idx = self.sparse_label[head]["label2idx"][item[head]]
                one_hot = [1 if i == idx else 0 for i in range(self.sparse_column[head]["num"])]
                feat.extend(one_hot)
            else:
                feat.append(item[head])
        return {"feat": torch.tensor(feat), "price": item["SalePrice"]}


def collate_function(data):
    feat = [datum["feat"] for datum in data]
    feat = torch.stack(feat).float()
    price = [datum["price"] for datum in data]
    price = torch.tensor(price).float()
    return {"feat": feat, "price": price}

In [ ]:
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage: Optional[str] = None) -> None:
        dataset = MyDataset()
        self.train_dataset, self.val_dataset = random_split(dataset, [1260, 200])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, 256, True, collate_fn=collate_function)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, 256, False, collate_fn=collate_function)

In [ ]:
dataset = MyDataset()

In [ ]:
dataset[0]["feat"].size()

In [ ]:
class LinearModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.add_module("",nn.Linear(7,7))
        self.fc1 = nn.Linear(304, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 32)
        self.fc6 = nn.Linear(32, 32)
        self.fc7 = nn.Linear(32, 1)

        self.mse = MeanSquaredError()
        self.r2 = R2Score()

    def forward(self, feat):
        x = F.relu(self.fc1(feat))
        x = F.relu(self.fc2(x)) + x
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x)) + x
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x)) + x
        x = self.fc7(x)
        x = x.squeeze(dim=1)
        return x

    def training_step(self, batch, batch_idx):
        predict = self(batch["feat"])
        loss = F.mse_loss(predict, batch["price"])
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        total_loss = 0
        for output in outputs:
            total_loss += output["loss"].item()

        logger = self.logger.experiment
        logger.add_scalar("train loss", total_loss, self.current_epoch + 1)

    def validation_step(self, batch, batch_idx):
        predict = self(batch["feat"])
        self.mse(predict, batch["price"])
        self.r2(predict, batch["price"])

    def validation_epoch_end(self, outputs):
        mse = self.mse.compute()
        r2 = self.r2.compute()
        logger = self.logger.experiment
        logger.add_scalar("mse", mse, self.current_epoch + 1)
        logger.add_scalar("r2", r2, self.current_epoch + 1)
        self.mse.reset()
        self.r2.reset()

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-4)

In [ ]:
model = LinearModel()

In [ ]:
logger = TensorBoardLogger('logs', name='linear')
checkpoint = ModelCheckpoint(dirpath="./checkpoints/linear", save_top_k=-1, every_n_epochs=20)
trainer = pl.Trainer(gpus=1,
                     max_epochs=200,
                     logger=logger,
                     callbacks=[checkpoint],
                     reload_dataloaders_every_n_epochs=5)

In [ ]:
trainer.fit(model, datamodule=MyDataModule())

# 测试

In [ ]:
class PredictSet(Dataset):
    def __init__(self):
        self.csv = pd.read_csv("./data/test.csv")
        with open("./data/sparse_label.json", "r", encoding="utf-8") as f:
            self.sparse_label = json.loads(f.read())
        with open("./data/sparse_column.json", "r", encoding="utf-8") as f:
            self.sparse_column = json.loads(f.read())

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, index):
        item = self.csv.iloc[index]
        feat = []
        for head in self.csv:
            if head == "Id":
                continue
            if head in self.sparse_label:
                idx = self.sparse_label[head]["label2idx"][item[head]]
                one_hot = [1 if i == idx else 0 for i in range(self.sparse_column[head]["num"])]
                feat.extend(one_hot)
            else:
                feat.append(item[head])
        return {"feat": torch.tensor(feat), "idx": item["Id"]}


def collate_fn(data):
    feat = [datum["feat"] for datum in data]
    feat = torch.stack(feat).float()
    idx = [datum["idx"] for datum in data]
    return {"feat": feat, "id": idx}

In [ ]:
predict_set = PredictSet()
predict_loader = DataLoader(predict_set, 256, False, collate_fn=collate_fn)

In [ ]:
model.cuda()

In [ ]:
test_result = []
with torch.no_grad():
    for batch in predict_loader:
        predict = model(batch["feat"].cuda())
        for price, id in zip(predict, batch["id"]):
            test_result.append({"Id": id, "SalePrice": price.item()})

In [ ]:
result_df = pd.DataFrame(test_result)

In [ ]:
result_df.to_csv("./data/result4.csv")